In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

import plotly
import plotly.graph_objs as go
import plotly.io as pio

In [14]:
# Load dataframes from csv files
census_df = pd.read_csv('df_census.csv', index_col=0)
ec_df = pd.read_csv('df_ec.csv', index_col=0)
election_df = pd.read_csv('df_election.csv', index_col=0)
abbrev_df = pd.read_csv('df_abbrev.csv', index_col=0)

In [39]:
print(census_df.loc["United States", census_df.columns[0]])
print(sum(census_df.loc["Alabama":"Wyoming", "HC01_VC03"]))
print(type(census_df.loc["Alabama":"Wyoming", "HC01_VC03"]))
census_df.loc[:, "HC01_VC03"].dtype
ec_df["Number of Electoral Votes"].dtype

325719178
325719178
<class 'pandas.core.series.Series'>


dtype('int64')

In [48]:
# Calculate Average voting power (EV share PER person)
avg_vp = ec_df["Number of Electoral Votes"].sum() / float(census_df.loc["United States", "HC01_VC03"])
print('Average share of an electoral vote for an individual:\n{:.2e}'
      .format(avg_vp))

# Store inverse
people_per_ev = 1.0/avg_vp
print('\nAverage amount of people per electoral vote:\n{:.2f}'.format(people_per_ev))

Average share of an electoral vote for an individual:
1.65e-06

Average amount of people per electoral vote:
605425.98


### I. Electoral Vote Apportionment Bar Chart

blahblah

In [64]:
ec_df.loc['Alabama']['Number of Electoral Votes']

9

In [199]:
# Get list of states in decreasing order of population
states = sorted(list(census_df.index[1:]), 
                key=lambda state: census_df.loc[state, 'HC01_VC03'],
                reverse=True)

state_pop = go.Bar(
    x=list(range(51)),#states,
    y=[census_df.loc[state, 'HC01_VC03'] for state in states],
    name='Population',
    marker=dict(
        color='#C2EFEB',
        #opacity=,
    )
)

ev_color = '#6EA4BF'
state_ev = go.Bar(
    x=list(range(51)),#states,
    y=[ec_df.loc[state, 'Number of Electoral Votes'] for state in states],
    name='Electoral Votes',
    yaxis='y2',
    #offset=4,
    marker=dict(
        color=ev_color,
        #opacity=,
    ),
    width=0.2,
)

# Hacky way to get grouped bars on multiple y-axes. Not working.
dummies = [
    go.Bar(
        x=['Alabama'], 
        y=[0],
        yaxis='y',
        name='y dummy', 
        hoverinfo='none', 
        showlegend=False
    ), 
    go.Bar(
        x=['Alabama'],
        y=[0],
        yaxis='y2', 
        name='y2 dummy', 
        hoverinfo='none', 
        showlegend=False
    ),
]

b_top = census_df.loc['California', 'HC01_VC03']
b_bot = 33300000
b_x = 1.2
b_size = 1.0
bracket = go.Scatter(
    x=[b_x, b_x+b_size, b_x+b_size, b_x],
    y=[b_bot, b_bot, b_top, b_top],
    fill=None,
    mode='lines',
    line=dict(
        width=1.0,
        color='black',
    ),
)

data = [state_pop, state_ev] + [bracket] #+ dummies

layout = go.Layout(
    title=dict(   
        text='State populations and electoral votes are NOT proportional',
        font=dict(
            family='verdana',
            size=18,
            color=None,
        ),
        #xref='paper',
        yref='paper',
        y=1.0,
        yanchor='bottom',
        #xanchor='right',
        pad=dict( # unit: pixels
            b=20,
        ),
    ),
    width=900,
    height=500,
    #barmode='group',
    showlegend=False,
    xaxis=dict(
        tickfont=dict(
            size=10,
        ),
        tickangle=-65,
        #automargin=True,
        tickmode='array',
        tickvals=list(range(51)),
        ticktext=states[:states.index('District of Columbia')] \
                 + ['D.C.'] + states[states.index('District of Columbia')+1:] 
    ),
    yaxis=dict(
        title='Population',
        range=[0,40000000],
        tickfont=dict(
            size=12,
        ),
        rangemode='tozero',
    ),
    yaxis2=dict(
        title='Electoral Votes',
        range=[0,40000000/float(people_per_ev)],
        color=ev_color,
#         titlefont=dict(
#             color=ev_color,
#         ),
        tickfont=dict(
            size=12,
        ),
        overlaying='y',
        side='right',
        rangemode='tozero',
        zeroline=False,
    )
)

notes = [dict(
    text='<i>This gap represents population<br>' +\
         'not represented by electoral votes</i>',
    font=dict(
        family=None,
        size=12,
        color='black',
    ),
    x=b_x + b_size,
    y=(b_top + b_bot)/2.0,
    xref='x',
    yref='y',
    align='left',
    xanchor='left',
    yanchor='middle',
    showarrow=True,
    arrowcolor='black',
    arrowwidth=1.2,
    arrowhead=0,
    arrowsize=1,
    startstandoff=3,
    axref='x',
    ayref='y',
    ax=6,
    ay=(b_top + b_bot)/2.0,
    xshift=0,
)]

layout.update(annotations=notes)

fig = go.Figure(data=data, layout=layout)
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot(fig)

In [201]:
pio.write_image(fig, 'images/ec_bars01.png', format='png', scale=3.0)

In [202]:
# Get list of states in decreasing order of population
states = sorted(list(census_df.index[1:]), 
                key=lambda state: census_df.loc[state, 'HC01_VC03'],
                reverse=True)

bar_width = 0.36
state_pop = go.Bar(
    x=list(range(51)),#states,
    y=[census_df.loc[state, 'HC01_VC03'] for state in states],
    name='Population',
    marker=dict(
        color='#C2EFEB',
        #opacity=,
    ),
    width=bar_width
)

ev_color = '#6EA4BF'
state_ev = go.Bar(
    x=list(range(51)),#states,
    y=[ec_df.loc[state, 'Number of Electoral Votes'] for state in states],
    name='Electoral Votes',
    yaxis='y2',
    marker=dict(
        color=ev_color,
        #opacity=,
    ),
    width=bar_width,
    offset=bar_width/2.0,
)

b_top = census_df.loc['California', 'HC01_VC03']
b_bot = 33300000
b_x = 1.2
b_size = 1.0
bracket = go.Scatter(
    x=[b_x, b_x+b_size, b_x+b_size, b_x],
    y=[b_bot, b_bot, b_top, b_top],
    fill=None,
    mode='lines',
    line=dict(
        width=1.0,
        color='black',
    ),
)

data = [state_pop, state_ev] + [bracket]

layout = go.Layout(
    title=dict(   
        text='State populations and electoral votes are NOT proportional',
        font=dict(
            family='verdana',
            size=18,
            color=None,
        ),
        #xref='paper',
        yref='paper',
        y=1.0,
        yanchor='bottom',
        #xanchor='right',
        pad=dict( # unit: pixels
            b=20,
        ),
    ),
    width=900,
    height=500,
    barmode='group',
    showlegend=False,
    xaxis=dict(
        tickfont=dict(
            size=10,
        ),
        tickangle=-65,
        #automargin=True,
        tickmode='array',
        tickvals=list(range(51)),
        ticktext=states[:states.index('District of Columbia')] \
                 + ['D.C.'] + states[states.index('District of Columbia')+1:] 
    ),
    yaxis=dict(
        title='Population',
        range=[0,40000000],
        tickfont=dict(
            size=12,
        ),
        rangemode='tozero',
    ),
    yaxis2=dict(
        title='Electoral Votes',
        range=[0,40000000/float(people_per_ev)],
        color=ev_color,
#         titlefont=dict(
#             color=ev_color,
#         ),
        tickfont=dict(
            size=12,
        ),
        overlaying='y',
        side='right',
        rangemode='tozero',
        zeroline=False,
    )
)

notes = [dict(
    text='<i>This gap represents population<br>' +\
         'not represented by electoral votes</i>',
    font=dict(
        family=None,
        size=12,
        color='black',
    ),
    x=b_x + b_size,
    y=(b_top + b_bot)/2.0,
    xref='x',
    yref='y',
    align='left',
    xanchor='left',
    yanchor='middle',
    showarrow=True,
    arrowcolor='black',
    arrowwidth=1.2,
    arrowhead=0,
    arrowsize=1,
    startstandoff=3,
    axref='x',
    ayref='y',
    ax=6,
    ay=(b_top + b_bot)/2.0,
    xshift=0,
)]

layout.update(annotations=notes)

fig = go.Figure(data=data, layout=layout)
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot(fig)

In [203]:
pio.write_image(fig, 'images/ec_bars02.png', format='png', scale=3.0)

### II. US Map by Voting Power

blahblah

In [316]:
# Try out function to convert matplotlib colormaps to Plotly format
import matplotlib
from matplotlib import cm
import numpy as np

magma_cmap = matplotlib.cm.get_cmap('PiYG_r')

def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k*h)[:3])*255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

magma = matplotlib_to_plotly(magma_cmap, 255)
#magma[123:132]
modm = magma[:123] + [[0.49, 'rgb(255, 255, 255)'],[0.51, 'rgb(255, 255, 255)']] + magma[132:]
modm[120:126]

[[0.47244094488188976, 'rgb(242, 246, 235)'],
 [0.4763779527559055, 'rgb(242, 246, 237)'],
 [0.48031496062992124, 'rgb(243, 246, 238)'],
 [0.49, 'rgb(255, 255, 255)'],
 [0.51, 'rgb(255, 255, 255)'],
 [0.5196850393700787, 'rgb(248, 242, 245)']]

In [422]:
# Calculate State Voting Power (SVP) and normalize by average voting power
states_vp = [ec_df.loc[state, "Number of Electoral Votes"] / \
             float(census_df.loc[state, "HC01_VC03"]) for state in states]
norm_states_vp = [vp/avg_vp for vp in states_vp]

rasp_green = [
    [0.0, 'rgb(144,11,85)'],
    #[0.4, 'blue'],
    [0.495, 'white'],
    [0.505, 'white'],
    [0.7, 'rgb(43,102,30)'],
    [1.0, 'rgb(43,102,30)']
]

orange_green = [
    [0.0, 'rgb(130,61,17)'],
    [0.25, 'rgb(218,126,38)'],
    [0.495, 'white'],
    [0.505, 'white'],
    [0.7, 'rgb(4,62,50)'],
    [1.0, 'rgb(4,62,50)']
]
# Plot
vp_map = go.Choropleth(
    colorscale=rasp_green,#'RdBu',
    autocolorscale=False,
    reversescale=True,
    locations=[abbrev_df.loc[state, 'Postal Code'] for state in states], # Needs abbreviations
    z=norm_states_vp,
    locationmode='USA-states',
    marker=dict(
        line=dict(
            color='black',
            width=1.0,
        )
    ),
    zauto=True,
    zmid=1.0,
    zmin=min(norm_states_vp),
    zmax=max(norm_states_vp),
    colorbar = dict(
        title = "Voting Power"
    )
)

data = [vp_map]

layout = go.Layout(
    title=go.layout.Title(
        text='Wooooooooooo'
    ),
    width=900,
    height=600,
    geo=go.layout.Geo(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True,
        lakecolor='rgb(255, 255, 255)'),
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot(fig)

In [423]:
pio.write_image(fig, 'images/ec_vp_map01.png', format='png', scale=3.0)

In [310]:
rasp_green[:2] + [[0.4, 'rgb(12,12,2)']] + rasp_green[3:]

[[0.0, 'rgb(144,11,85)'],
 [0.495, 'white'],
 [0.4, 'rgb(12,12,2)'],
 [0.7, 'rgb(43,102,30)'],
 [1.0, 'rgb(43,102,30)']]

In [435]:
RdPu_cmap = matplotlib.cm.get_cmap('RdPu')
RdPu = matplotlib_to_plotly(RdPu_cmap, 255)

# Plot
vp_map = go.Choropleth(
    colorscale=RdPu,
    autocolorscale=False,
    reversescale=False,
    locations=[abbrev_df.loc[state, 'Postal Code'] for state in states], # Needs abbreviations
    z=norm_states_vp,
    locationmode='USA-states',
    marker=dict(
        line=dict(
            color='black',
            width=1.0,
        )
    ),
    zauto=False,
    zmin=min(norm_states_vp),
    zmax=max(norm_states_vp),
    colorbar = dict(
        title = "Voting Power"
    ),
    geo='geo2',
)

whiteness = [census_df.loc[state, 'HC03_VC99'] for state in states]
w_map = go.Choropleth(
    colorscale=RdPu,
    autocolorscale=False,
    reversescale=False,
    locations=[abbrev_df.loc[state, 'Postal Code'] for state in states], # Needs abbreviations
    z=whiteness,
    locationmode='USA-states',
    marker=dict(
        line=dict(
            color=['black']*36 + ['yellow']*2 + ['black']*1 + ['yellow']*12 ,
            width=[1.0]*36 + [2.0]*2 + [1.0]*1 + [2.0]*12,
        )
    ),
    zauto=False,
    zmin=min(whiteness)+20,
    zmax=max(whiteness),
    colorbar = dict(
        title = "Whiteness"
    ),
    geo='geo',
)

data = [vp_map, w_map]

layout = go.Layout(
    title=go.layout.Title(
        text='Wooooooooooo'
    ),
    width=900,
    height=900,
    grid=dict(
        rows=2,
        columns=1
    ),
    geo=go.layout.Geo(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True,
        lakecolor='rgb(255, 255, 255)',
        domain=dict(
            row=0,
            column=0,
        )
    ),
    geo2=go.layout.Geo(
        scope='usa',
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True,
        lakecolor='rgb(255, 255, 255)',
        domain=dict(
            row=1,
            column=0,
        )
    ),
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot(fig)

In [425]:
pio.write_image(fig, 'images/ec_maps_01.png', format='png', scale=3.0)

In [349]:
for i in range(50, -1, -1):
    print('{:.2f} {}'.format(norm_states_vp[i], states[i]))

3.14 Wyoming
2.91 Vermont
2.62 District of Columbia
2.46 Alaska
2.40 North Dakota
2.09 South Dakota
1.89 Delaware
1.73 Montana
2.29 Rhode Island
1.81 Maine
1.80 New Hampshire
1.70 Hawaii
1.41 Idaho
1.67 West Virginia
1.58 Nebraska
1.45 New Mexico
1.25 Kansas
1.22 Mississippi
1.21 Nevada
1.21 Arkansas
1.17 Utah
1.15 Iowa
1.18 Connecticut
1.08 Oklahoma
1.02 Oregon
1.09 Kentucky
1.03 Louisiana
1.12 Alabama
1.08 South Carolina
1.09 Minnesota
0.97 Colorado
1.04 Wisconsin
1.00 Maryland
0.99 Missouri
1.00 Indiana
0.99 Tennessee
0.97 Massachusetts
0.95 Arizona
0.98 Washington
0.93 Virginia
0.94 New Jersey
0.97 Michigan
0.88 North Carolina
0.93 Georgia
0.93 Ohio
0.95 Illinois
0.95 Pennsylvania
0.88 New York
0.84 Florida
0.81 Texas
0.84 California


### IV. People Plot

In [450]:
dummy = go.Scatter(
    x = [0],
    y = [0],
    mode = 'none'
)

data = [dummy]

layout = go.Layout(
    title=dict(   
        text='Vote loss/gain by racial groups',
        font=dict(
            family='verdana',
            size=18,
            color=None,
        ),
        #xref='paper',
        yref='paper',
        y=1.0,
        yanchor='bottom',
        #xanchor='right',
        pad=dict( # unit: pixels
            b=20,
        ),
    ),
     width=900,
     height=640,
    showlegend=False,
    xaxis=dict(
        range=[0,45],
        tickfont=dict(
            size=10,
        ),
        tick0=0,
        dtick=1.0,
        constrain='domain',
    ),
    yaxis=dict(
        title='Population',
        range=[0, 100],
        scaleanchor='x',
        scaleratio=1,
        tickfont=dict(
            size=12,
        ),
        tick0=0,
        dtick=1.0,
        constrain='domain',
    ),
)

split = 3
asplit = 2
vshift = (22-split)/2.0/2.0   # same as foot
foot = vshift   # radius
hand = (10-asplit)/2.0   # radius
head = 8   # radius
neck = 1

color = 'black'
bgcolor = 'white'

man = [
    # head
    dict(
        type='circle',
        x0=21-head,
        y0=75.5+vshift+neck,
        x1=21+head,
        y1=75.5+(2*head)+vshift+neck,
        fillcolor=color,
        line=dict(
            width=0,
        )
    ),
    # body
    dict(
        type='rect',
        x0=10,
        y0=vshift,
        x1=32,
        y1=75.5+vshift,
        fillcolor=color,
        line=dict(
            width=0,
        )
    ),
    # arms
    dict(
        type='rect',
        x0=0,
        y0=37.5+hand+vshift,
        x1=42,
        y1=65.5+vshift,
        fillcolor=color,
        line=dict(
            width=0,
        )
    ),
    # left shoulder
    dict(
        type='circle',
        x0=0,
        y0=55.5+vshift,
        x1=20,
        y1=75.5+vshift,
        fillcolor=color,
        line=dict(
            width=0,
        )
    ),
    # right shoulder
    dict(
        type='circle',
        x0=22,
        y0=55.5+vshift,
        x1=42,
        y1=75.5+vshift,
        fillcolor=color,
        line=dict(
            width=0,
        )
    ),
    # leg split
    dict(
        type='rect',
        x0=21-split/2.0,
        y0=vshift,
        x1=21+split/2.0,
        y1=37.5+vshift,
        fillcolor=bgcolor,
        line=dict(
            width=0,
        ),
    ),
    # left arm split
    dict(
        type='rect',
        x0=10-asplit,
        y0=37.5+hand+vshift,
        x1=10,
        y1=64.5+vshift,
        fillcolor=bgcolor,
        line=dict(
            width=0,
        ),
    ),
    # right arm split
    dict(
        type='rect',
        x0=32,
        y0=37.5+hand+vshift,
        x1=32+asplit,
        y1=64.5+vshift,
        fillcolor=bgcolor,
        line=dict(
            width=0,
        ),
    ),
    # left hand
    dict(
        type='circle',
        x0=0,
        y0=37.5+vshift,
        x1=2*hand,
        y1=37.5+(2*hand)+vshift,
        fillcolor=color,
        line=dict(
            width=0,
        ),
    ),
    # right hand
    dict(
        type='circle',
        x0=32+asplit,
        y0=37.5+vshift,
        x1=42,
        y1=37.5+(2*hand)+vshift,
        fillcolor=color,
        line=dict(
            width=0,
        ),
    ),
    # left foot
    dict(
        type='circle',
        x0=10,
        y0=0,
        x1=10+(2*foot),
        y1=2*vshift,
        fillcolor=color,
        line=dict(
            width=0,
        ),
    ),
    # right foot
    dict(
        type='circle',
        x0=32-(2*foot),
        y0=0,
        x1=32,
        y1=2*vshift,
        fillcolor=color,
        line=dict(
            width=0,
        ),
    ),
]

layout['shapes'] = man

fig = go.Figure(data=data, layout=layout)
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot(fig)

In [203]:
pio.write_image(fig, 'images/ec_bars02.png', format='png', scale=3.0)

In [427]:
4**.5

2.0